In [55]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
from skimage import io, transform
import torchvision.transforms as transforms
from torch.autograd import Variable
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import random
import math
import torch.nn.functional as F


# Dataset Loading Multiple Folder

In [56]:
# Data loading code
traindir = '/work/test/Desktop/python/pytoch/dataset/'

normalize = transforms.Normalize(mean=[0.0, 0.0, 0.0],
                                 std=[1, 1, 1])

train_dataset = dsets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(28),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))

# Hyperparameter to divide train dataset

In [57]:
batch_size = 1
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Divide dataset into train and validate

In [53]:
from torch.utils.data.sampler import SubsetRandomSampler

# Creating data indices for training and validation splits:
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                                sampler=valid_sampler)


# If we dont want want to validate

In [15]:
"""train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=1, 
        num_workers=4, pin_memory=True)"""

# Size of test and validate Dataset

In [65]:
print(len(train_loader))
print(len(validation_loader))

1081
270


# Create Model with optimizer and loss function

In [61]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 5, 1)
        self.batch_1=nn.BatchNorm2d(20)
        self.conv2=nn.Conv2d(20, 50, 5, 1)
        self.conv3=nn.Conv2d(50,60,5,1)
        self.fc1 = nn.Linear(2*2*60, 500)
        #self.batch_2=nn.BatchNorm1d(500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = self.batch_1(x)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 2*2*60)
        x = F.relu(self.fc1(x))
        #x = self.batch_2(x)
        x = self.fc2(x)
        return x

model=CNN()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

criteria=nn.CrossEntropyLoss()


# Train and Test model

In [63]:
#test the model
for epoch in range(35):
    ave_loss=0.0
    for i, (input, target) in enumerate(train_loader):
        #print(input.shape,target)
        optimizer.zero_grad()
        output=model(input)

        loss=criteria(output,target)
        loss.backward()
        optimizer.step()
        ave_loss+=loss.item()
        if i%200==199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, ave_loss / 100))
            ave_loss = 0.0
correct=0
total=0


#test the model
with torch.no_grad():
    for inputs,labels in validation_loader:
#         inputs=data['image']
#         labels=data['label']
        
        inputs=torch.from_numpy(np.array(inputs))
        labels=torch.from_numpy(np.array(labels))
        outputs=model(inputs)
        _,predicted=torch.max(outputs.data,1)
        total+=labels.size(0)
        correct+=(predicted==labels).sum().item()

print('Accuracy of the network on the  test images: %0.6f %%' % (
    100 * correct / total))

[1,   200] loss: 1.967
[1,   400] loss: 2.245
[1,   600] loss: 2.382
[1,   800] loss: 2.168
[1,  1000] loss: 1.815
[2,   200] loss: 1.895
[2,   400] loss: 1.996
[2,   600] loss: 2.167
[2,   800] loss: 2.186
[2,  1000] loss: 2.028
[3,   200] loss: 2.112
[3,   400] loss: 2.127
[3,   600] loss: 1.973
[3,   800] loss: 2.082
[3,  1000] loss: 1.940
[4,   200] loss: 2.077
[4,   400] loss: 2.019
[4,   600] loss: 1.946
[4,   800] loss: 1.908
[4,  1000] loss: 2.219
[5,   200] loss: 2.061
[5,   400] loss: 1.910
[5,   600] loss: 1.909
[5,   800] loss: 1.966
[5,  1000] loss: 2.132
[6,   200] loss: 1.982
[6,   400] loss: 1.988
[6,   600] loss: 2.214
[6,   800] loss: 2.275
[6,  1000] loss: 1.914
[7,   200] loss: 2.511
[7,   400] loss: 2.051
[7,   600] loss: 1.973
[7,   800] loss: 2.028
[7,  1000] loss: 1.562
[8,   200] loss: 1.960
[8,   400] loss: 2.017
[8,   600] loss: 1.786
[8,   800] loss: 2.272
[8,  1000] loss: 2.279
[9,   200] loss: 2.034
[9,   400] loss: 2.178
[9,   600] loss: 1.912
[9,   800] 